# MLflow Project
Вам предстоит реализовать модель классификации цветков Ириса в виде MLflow проекта. Для создания MLflow проекта нужна минимум 2 вещи:

*   Python модуль, реализующий эксперимент
*   `MLProject` файл, содержащий ифнормацию об эксперименте

Но для начала установим `mlflow` в Collab

# Новый раздел

In [ ]:
import mlflow

In [ ]:
!pip install --quiet "mlflow[extras]"

     |████████████████████████████████| 14.2MB 318kB/s 
     |████████████████████████████████| 153kB 40.7MB/s 
     |████████████████████████████████| 1.1MB 42.7MB/s 
     |████████████████████████████████| 163kB 42.1MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 133kB 43.9MB/s 
     |████████████████████████████████| 2.2MB 36.0MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 1.8MB 41.8MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 7.3MB 31.5MB/s 
     |████████████████████████████████| 215kB 43.5MB/s 
     |████████████████████████████████| 727kB 30.4MB/s 
     |████████████████████████████████| 1.1MB 40.3MB/s 
     |

# Новый раздел

In [ ]:
!mlflow run -e load_raw_data --no-conda --experiment-name "Movielens recommendations" .

INFO: 'Movielens recommendations' does not exist. Creating a new experiment
2021/03/08 19:37:26 INFO mlflow.projects.utils: === Created directory /tmp/tmpxx5ikinc for downloading remote URIs passed to arguments of type 'path' ===
2021/03/08 19:37:26 INFO mlflow.projects.backend.local: === Running command 'python load_raw_data.py \
  --url=http://files.grouplens.org/datasets/movielens/ml-latest-small.zip 
' in run with ID 'df81728ba50e4e5e8ca6e83888e8113f' === 
2021/03/08 19:37:28 INFO mlflow.projects: === Run (ID 'df81728ba50e4e5e8ca6e83888e8113f') succeeded ===


In [ ]:
client = mlflow.tracking.MlflowClient()
client.search_runs(experiment_ids="0")

[]

In [ ]:
experiment = client.get_experiment_by_name("Movielens recommendations")

In [ ]:
experiment

<Experiment: artifact_location='file:///content/mlruns/1', experiment_id='1', lifecycle_stage='active', name='Movielens recommendations', tags={}>

In [ ]:
mlflow.get_artifact_uri()

'file:///content/mlruns/0/d1824c5c739c496c96af84b678136f46/artifacts'

In [ ]:
!mlflow run -e etl_data --no-conda --experiment-name "Movielens recommendations" -P ratings_csv='/content/mlruns/1/df81728ba50e4e5e8ca6e83888e8113f/artifacts/ratings.csv' .

2021/03/08 19:37:53 INFO mlflow.projects.utils: === Created directory /tmp/tmpdr062t1y for downloading remote URIs passed to arguments of type 'path' ===
2021/03/08 19:37:53 INFO mlflow.projects.backend.local: === Running command 'python etl_data.py \
  --ratings_csv=/content/mlruns/1/df81728ba50e4e5e8ca6e83888e8113f/artifacts/ratings.csv \
  --max_row_limit=10000 
' in run with ID '9569bfdeab9e458891dd74be26f1cdd4' === 
2021/03/08 19:37:55 INFO mlflow.projects: === Run (ID '9569bfdeab9e458891dd74be26f1cdd4') succeeded ===


In [ ]:
!mlflow run -e nmf --no-conda --experiment-name "Movielens recommendations" -P ratings_data='/content/mlruns/1/9569bfdeab9e458891dd74be26f1cdd4/artifacts/df.parquet.gzip' .

2021/03/08 19:38:14 INFO mlflow.projects.utils: === Created directory /tmp/tmp1_w2ntc_ for downloading remote URIs passed to arguments of type 'path' ===
2021/03/08 19:38:14 INFO mlflow.projects.backend.local: === Running command 'python nmf.py \
  --ratings_data=/content/mlruns/1/9569bfdeab9e458891dd74be26f1cdd4/artifacts/df.parquet.gzip \
  --split_prop=0.8 \
  --max_iter=100 \
  --alpha=0.1 \
  --n_components=12
' in run with ID 'd139cbef90724925a05f938db5ad3f2b' === 
2021/03/08 19:38:17 INFO mlflow.projects: === Run (ID 'd139cbef90724925a05f938db5ad3f2b') succeeded ===


In [ ]:
!mlflow run -e xgboost_model --no-conda --experiment-name "Movielens recommendations" -P ratings_data='/content/mlruns/1/9569bfdeab9e458891dd74be26f1cdd4/artifacts/df.parquet.gzip' -P als_model_uri='/content/mlruns/1/d139cbef90724925a05f938db5ad3f2b/artifacts/models/' .

2021/03/08 19:51:38 INFO mlflow.projects.utils: === Created directory /tmp/tmpfdbmg4nf for downloading remote URIs passed to arguments of type 'path' ===
2021/03/08 19:51:38 INFO mlflow.projects.backend.local: === Running command 'python xgboost_model.py \
  --ratings_data=/content/mlruns/1/9569bfdeab9e458891dd74be26f1cdd4/artifacts/df.parquet.gzip \
  --als_model_uri=/content/mlruns/1/d139cbef90724925a05f938db5ad3f2b/artifacts/models/ \
  --eta=0.3 \
  --max_depth=6 \
  --num_rounds=10' in run with ID 'e4d3dbbcfdf84b51b5d00456dc25170a' === 
2021/03/08 19:51:41 INFO mlflow.projects: === Run (ID 'e4d3dbbcfdf84b51b5d00456dc25170a') succeeded ===


In [ ]:
!mlflow run -e main --no-conda --experiment-name "Movielens recommendations" .

2021/03/08 20:21:52 INFO mlflow.projects.utils: === Created directory /tmp/tmpf81bro2_ for downloading remote URIs passed to arguments of type 'path' ===
2021/03/08 20:21:52 INFO mlflow.projects.backend.local: === Running command 'python main.py \
  --url=http://files.grouplens.org/datasets/movielens/ml-latest-small.zip ' in run with ID 'ef8d2765b88b4c2dabeeeccdf617dbb8' === 
2021/03/08 20:21:56 INFO mlflow.projects: === Run (ID 'ef8d2765b88b4c2dabeeeccdf617dbb8') succeeded ===


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 747kB 9.3MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.3-cp37-none-any.whl size=18979 sha256=d13cc1320402a56eecec73bd803d8feae6fa76466ab31704a9456785e7e8c005
  Stored in directory: /root/.cache/pip/wheels/f0/77/01/d23761888c52099d1817121dc1f28afbc1c7cbf3caac93b1ff
Successfully built pyngrok


In [ ]:
H = pymodel.components_
      dtrain = xgb.DMatrix(pymodel.transform(X_train), label=X_train_orig.rating)
      dtest = xgb.DMatrix(pymodel.transform(X_test_fixed), label=X_test_orig.rating)
      
      params = {
            "objective": "reg:squarederror",
            "max_depth ": args.max_depth ,
            "eta": args.eta,
            "seed": 42,
        }
      model = xgb.train(params, dtrain, args.num_rounds)

      # evaluate model
      y_proba = model.predict(dtest)
      y_pred = y_proba.argmax(axis=1)
      mse = mean_squared_error(X_test_orig.rating, model.predict(dtest))
